In [1]:
import time

In [2]:
file1 = open("dna.txt","r") 
file2 = open("patterns.txt","r")

dnaContents = ''
patternsContents = []

for line in file1.readlines():
    line = line.strip()
    dnaContents = line
    
for line in file2.readlines():
    line = line.strip()
    patternsContents.append(line)

In [3]:
print(dnaContents)
print(patternsContents)

AAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
['AAA', 'AGCG', 'AGCGTA', 'GATA']


Write a Python program/notebook that reads two text files. One file is a given DNA/protein sequence and
the second file as a list of patterns to search for. Each pattern in a line. Implement the algorithms below.
Your program should report all occurrences of the patterns and the time it took to find them. Your
program should find the following:

1. (25 pnts) Boyer-Moore.

In [4]:
def charMismatch(string, size): 
    mismatchChar = [-1]*256 
    for i in range(size): 
        mismatchChar[ord(string[i])] = i; 
    return mismatchChar

def searchBMA(text, pattern): 
    
    pLen = len(pattern) 
    tLen = len(text)

    mismatch = charMismatch(pattern, pLen)  
    shift = 0
    while(shift <= tLen-pLen): 
        j = pLen-1
  
        while j>=0 and pattern[j] == text[shift+j]: 
            j -= 1

        if j<0: 
            print("Found pattern at index {}".format(shift)) 
            shift += (pLen-mismatch[ord(text[shift+pLen])] if ((shift+pLen)<tLen) else 1) 
        else: 
            shift += max(1, j-mismatch[ord(text[shift+j])])


def main(): 
    print("Boyer-Moore Algorithm results: \n")
    
    start = time.perf_counter()
    for pattern in patternsContents:
        print("Results for pattern:", pattern)
        searchBMA(dnaContents, pattern)
        print("\n")
    end = time.perf_counter()
    print("time is:", end-start, "seconds")
    
if __name__ == '__main__': 
    main() 

Boyer-Moore Algorithm results: 

Results for pattern: AAA
Found pattern at index 0
Found pattern at index 46
Found pattern at index 51
Found pattern at index 74


Results for pattern: AGCG
Found pattern at index 98


Results for pattern: AGCGTA


Results for pattern: GATA
Found pattern at index 11
Found pattern at index 110


time is: 0.0011760999999999022 seconds


2. (25 pnts) KMP. 

In [5]:
def KMP(text, pattern): 
    M = len(pattern) 
    N = len(text) 

    listpro = [0]*M 
    j = 0

    listProcessing(pattern, M, listpro) 
  
    i = 0 
    while i < N: 
        if pattern[j] == text[i]: 
            i += 1
            j += 1
  
        if j == M: 
            print("Found pattern at index", str(i-j)) 
            j = listpro[j-1]  
        elif i < N and pattern[j] != text[i]: 
            if j != 0: 
                j = listpro[j-1] 
            else: 
                i += 1
                
def listProcessing(pattern, M, listpro): 
    suffixLength = 0

    listpro[0] = 0
    i = 1

    while i < M: 
        if pattern[i] == pattern[suffixLength]: 
            suffixLength += 1
            listpro[i] = suffixLength
            i += 1
        else: 
            if suffixLength != 0: 
                suffixLength = listpro[suffixLength-1] 
            else: 
                listpro[i] = 0
                i += 1

print("Knuth-Morris-Pratt Algorithm results: \n")
    
start = time.perf_counter()
    
for pattern in patternsContents:
    print("Results for pattern:", pattern)
    KMP(dnaContents, pattern)
    print("\n")

end = time.perf_counter()
print("time is:", end-start, "seconds")

Knuth-Morris-Pratt Algorithm results: 

Results for pattern: AAA
Found pattern at index 0
Found pattern at index 46
Found pattern at index 51
Found pattern at index 74


Results for pattern: AGCG
Found pattern at index 98


Results for pattern: AGCGTA


Results for pattern: GATA
Found pattern at index 11
Found pattern at index 110


time is: 0.0018131999999999593 seconds


3. (25 pnts) Suffix Arrays

In [11]:
suffixArray = sorted([dnaContents[i:] for i in range(len(dnaContents))])

start = time.perf_counter()
for pattern in patternsContents:
    print("Results for pattern", pattern)
    for i in suffixArray:
        if pattern in i in suffixArray:
            print("Found pattern at index:", dnaContents.index(pattern))
        break
    print("\n")
        
end = time.perf_counter()

print("time is:", end-start, "seconds")

Results for pattern AAA
Found pattern at index: 0


Results for pattern AGCG
Found pattern at index: 98


Results for pattern AGCGTA


Results for pattern GATA
Found pattern at index: 11


time is: 0.001462799999899289 seconds


4. (25 pnts) Suffix Trees.

In [12]:
class Node:
    def __init__(self, subTree="", childNode=[]):
        self.sub = subTree
        self.child = childNode
        
class SuffixTree:
    def __init__(self, treeString):
        self.nodes = [Node()]
        for i in range(len(treeString)):
            self.suffixAddition(treeString[i:])
 
    def suffixAddition(self, suffix):
        n = 0
        i = 0
        while i < len(suffix):
            bSuffix = suffix[i]
            x2 = 0
            while True:
                childrenNodes = self.nodes[n].child
                if x2 == len(childrenNodes):
                    n2 = len(self.nodes)
                    self.nodes.append(Node(suffix[i:], []))
                    self.nodes[n].child.append(n2)
                    return
                n2 = childrenNodes[x2]
                if self.nodes[n2].sub[0] == bSuffix:
                    break
                x2 = x2 + 1

            sub2 = self.nodes[n2].sub
            j = 0
            while j < len(sub2):
                if suffix[i + j] != sub2[j]:
                    n3 = n2
                    n2 = len(self.nodes)
                    self.nodes.append(Node(sub2[:j], [n3]))
                    self.nodes[n3].sub = sub2[j:]
                    self.nodes[n].child[x2] = n2
                    break
                j = j + 1
            i = i + j 
            n = n2 
 
    def visualize(self):
        if len(self.nodes) == 0:
            print("empty")
            return
 
        def display(x, pre):
            childrenNodes = self.nodes[x].child
            if len(childrenNodes) == 0:
                print("--", self.nodes[x].sub)
                return
            print("+-", self.nodes[x].sub)
            for ch in childrenNodes[:-1]:
                print(pre, "+-")
                display(ch, pre + " | ")
            print(pre, "+-")
            display(childrenNodes[-1], pre + "  ")
 
        display(0, "")

start = time.perf_counter()
SuffixTree(dnaContents).visualize()
end = time.perf_counter()
print("time is:", end-start, "seconds")

+- 
 +-
+- A
 |  +-
+- A
 |  |  +-
+- A
 |  |  |  +-
-- GAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  |  +-
+- T
 |  |  |    +-
+- T
 |  |  |    |  +-
-- AAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  |    |  +-
-- TTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  |    +-
-- ACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  +-
-- GAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  +-
+- C
 |  |  |  +-
-- TGGTTACCTGCCGTGAGTAAATTAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  |  +-
-- CAATATAGGCATAGCGCACAGACAGATAATAATTACAGAGTACACAACATCCAT
 |  |  |  +-
-- ATCCAT
 |  |  +-
+- T
 |  |    +-
+- T
 |  |    |  +-
+- A
 |  |    |  |  +-
-- AATTTTAT